# Introduction
We implement [TruthFinder](http://hanj.cs.illinois.edu/pdf/kdd07_xyin.pdf) in this notebook. There are three goals we want to achieve

1. Implement and integrate truthfinder into spectrum
2. Understand how they evaluate it
3. See how they prepare dataset

### How TruthFinder works

TruthFinder takes into inputs as a set of websites (sources), facts (claims), objects (things of interest). Its mission is to estimate the source reliability and truth facts.

**confidence of facts**: The confidence of a fact f is defined as the probability of f being correct.

**source reliability**: The reliability of a source s is defined as the expected confidence of facts provided by s.

We can visualize the relationship between sources, facts, and objects as shown below (Note that here the websites are the sources).

![](./gfx/truthfinder.png)

TruthFinder works as follows:
1. Initialize all source reliabilities to $t_0$
2. While $sim(t_{prev}, t_{now}) < threshold$:
    - compute fact confidences
    - compute website trustworthiness
    
here $t$ are the vector of all source reliabities and similarity function is the cosine similarity.

Now, in order to implement TruthFinder, we need to know how to compute fact confidences and website trustworthiness. We have the following formulas

$$s(f) = 1 -  \prod_{w \in W(f)}(1 -t(w))$$, where t(w) is computed as 

![](./gfx/truthfinder_source_reliability.png)

However we need to address to subtle problems with the above formula:

1. Numerical instability
2. Taking into account the influence among facts about the same object.

To address numerical instability, we will operate in log space (a common trick). Let's define the source reliabity score 
$$\tau(f) = -ln(1 - t(w))$$

and fact confidence score:

$$\sigma(f)= -ln(1 - s(f))$$.

It is easy to prove $$\sigma(f) = \sum_{w \in W(f)}\tau(w)$$

To account for the influence among facts about the same object, we introduce an adjusted confidence score 
![](./gfx/truthfinder_adjusted_score.png)


#### Implication between facts
$imp(f_1 \rightarrow f_2)$ is $f_1$'s influence on $f_{2}$'s confidence. This value is between -1 and 1. A positive value means they support each other and negative one means the opposite. This implication function is domain specific! and can be defined as 

$imp(f_1 \rightarrow f_2) = sim(f_1, f_2) - base\_sim$, where base_sim is a threshold.



![](./gfx/truthfinder_implication.png)

# Implementation

They compare source and object results.

In [ ]:
import pandas as pd
import os.path as op
import numpy as np
import seaborn as sns

In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
from spectrum import evaluator
from spectrum.judge.truthfinder import truthfinder

In [ ]:
DATA_DIR = '../data'
DATA_SET = 'population'

In [ ]:
truths = pd.read_csv(op.join(DATA_DIR, DATA_SET, 'truths.csv'))
claims = pd.read_csv(op.join(DATA_DIR, DATA_SET, 'claims.csv'))

In [ ]:
truths.head()

In [ ]:
claims.head()

In [ ]:
discovered_trusts, discovered_truths = truthfinder(claims, verbose=True, initial_trust=0.5)

In [ ]:
discovered_truths = discovered_truths.groupby('object_id').apply(lambda x: x.loc[x.confidence.idxmax()])[['value']]
discovered_truths.reset_index(inplace=True)

# Evaluation

In [ ]:
evaluator.accuracy(truths, discovered_truths)